# 离群点分析与异常检测

数据集：wine数据集  
来源：https://ir.library.oregonstate.edu/concern/datasets/47429f155?locale=en

In [1]:
from pyod.models.knn import KNN
import numpy as np
import pandas as pd
import os

/home/mqli/venv/lib/python3.6/site-packages/numba/errors.py:137: UserWarning: Insufficiently recent colorama version found. Numba requires colorama >= 0.3.9
  warnings.warn(msg)


## 数据集中所有csv文件数据合并 

In [3]:
domain = os.path.abspath(r'/home/mqli/data-mining/wine/benchmarks/')
wine_pd = pd.DataFrame()
columns = ['point.id', 'motherset', 'origin', 'original.label', 'diff.score', 'ground.truth',
           'fixed.acidity', 'volatile.acidity', 'citric.acid', 'residual.sugar', 'chlorides', 
           'free.sulfur.dioxide', 'total.sulfur.dioxide', 'density', 'pH', 'sulphates', 'alcohol']
i=0
for info in os.listdir('/home/mqli/data-mining/wine/benchmarks/'):
    info = os.path.join(domain,info)
    data = pd.read_csv(info)
    print(i)
    wine_pd = wine_pd.append(data, ignore_index = True)
    i += 1
print(wine_pd)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [4]:
wine_pd.to_csv("/home/mqli/data-mining/wine/benchmarks/all_file.csv",index=False)

In [2]:
wine_pd = pd.read_csv("/home/mqli/data-mining/wine/benchmarks/all_file.csv")

## 缺失值处理，全部用0填充 

In [3]:
wine_pd = wine_pd.fillna(0)
wine_pd

,point.id,motherset,origin,original.label,diff.score,ground.truth,fixed.acidity,volatile.acidity,citric.acid,residual.sugar,...,noise..24,noise..25,noise..26,noise..27,noise..28,noise..29,noise..30,noise..31,noise..32,noise..33
0,wine_point_2610,wine,regression,6,0.124411,nominal,-0.088942,-0.848330,-0.128224,-0.954902,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,wine_point_1054,wine,regression,4,0.095176,anomaly,1.685156,-0.301671,1.592143,-0.681666,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,wine_point_4490,wine,regression,6,0.038629,nominal,-0.397480,0.852387,1.179255,-0.534540,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,wine_point_1139,wine,regression,5,0.135267,anomaly,1.530886,-0.362411,0.972811,-0.471485,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,wine_point_2644,wine,regression,5,0.391649,anomaly,0.682405,-0.058711,-0.678741,-0.051123,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4184254,wine_point_4835,wine,regression,7,0.005481,nominal,-1.014558,0.184248,-0.128224,-0.828793,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4184255,wine_point_4438,wine,regression,6,0.130350,nominal,-0.551750,-0.301671,-0.128224,0.779092,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4184256,wine_point_4574,wine,regression,6,0.012012,nominal,-0.320346,-0.119451,-0.609926,-0.450467,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4184257,wine_point_2618,wine,regression,7,0.033862,nominal,-0.551750,-0.969809,0.628737,1.840506,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 处理标签数据（是否异常） 

In [4]:
wine_pd['ground.truth']=wine_pd['ground.truth'].map({'nominal':0,'anomaly':1})
wine_pd

,point.id,motherset,origin,original.label,diff.score,ground.truth,fixed.acidity,volatile.acidity,citric.acid,residual.sugar,...,noise..24,noise..25,noise..26,noise..27,noise..28,noise..29,noise..30,noise..31,noise..32,noise..33
0,wine_point_2610,wine,regression,6,0.124411,0,-0.088942,-0.848330,-0.128224,-0.954902,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,wine_point_1054,wine,regression,4,0.095176,1,1.685156,-0.301671,1.592143,-0.681666,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,wine_point_4490,wine,regression,6,0.038629,0,-0.397480,0.852387,1.179255,-0.534540,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,wine_point_1139,wine,regression,5,0.135267,1,1.530886,-0.362411,0.972811,-0.471485,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,wine_point_2644,wine,regression,5,0.391649,1,0.682405,-0.058711,-0.678741,-0.051123,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4184254,wine_point_4835,wine,regression,7,0.005481,0,-1.014558,0.184248,-0.128224,-0.828793,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4184255,wine_point_4438,wine,regression,6,0.130350,0,-0.551750,-0.301671,-0.128224,0.779092,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4184256,wine_point_4574,wine,regression,6,0.012012,0,-0.320346,-0.119451,-0.609926,-0.450467,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4184257,wine_point_2618,wine,regression,7,0.033862,0,-0.551750,-0.969809,0.628737,1.840506,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 使用pyod库中的PCA来训练异常检测模型

In [5]:
from sklearn.model_selection import train_test_split
from pyod.models.pca import PCA
from pyod.utils.data import evaluate_print
from pyod.utils.example import visualize
# 去掉数据中没用的列数据后将剩下的作为特征去训练
x = wine_pd.drop(['point.id','motherset','origin','ground.truth'],axis=1)
# 标签数据
y = np.array(wine_pd['ground.truth'])
random_seed = 5
# 切分数据集为训练集和测试集
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state=random_seed)
clf_name = 'PCA'
# 创建pca模型
clf = PCA()
# 训练模型
clf.fit(x_train)

# get the prediction labels and outlier scores of the training data
y_train_pred = clf.labels_  # binary labels (0: inliers, 1: outliers)
y_train_scores = clf.decision_scores_  # raw outlier scores

# get the prediction on the test data
y_test_pred = clf.predict(x_test)  # outlier labels (0 or 1)
y_test_scores = clf.decision_function(x_test)  # outlier scores

# evaluate and print the results
print("\nOn Training Data:")
evaluate_print(clf_name, y_train, y_train_scores)
print("\nOn Test Data:")
evaluate_print(clf_name, y_test, y_test_scores)

/home/mqli/venv/lib/python3.6/site-packages/sklearn/utils/validation.py:933: FutureWarning: Passing attributes to check_is_fitted is deprecated and will be removed in 0.23. The attributes argument is ignored.
  "argument is ignored.", FutureWarning)
/home/mqli/venv/lib/python3.6/site-packages/sklearn/utils/validation.py:933: FutureWarning: Passing attributes to check_is_fitted is deprecated and will be removed in 0.23. The attributes argument is ignored.
  "argument is ignored.", FutureWarning)



On Training Data:
PCA ROC:0.5779, precision @ rank n:0.129

On Test Data:
PCA ROC:0.5792, precision @ rank n:0.1294


从ROC指标来看，模型检测效果一般，有待提升，可能是选取的模型的原因，也有可能是特征选取的不够准确。于是尝试一下VAE模型来训练。

## VAE训练异常检测模型

In [7]:
from pyod.models.vae import VAE

contamination = 0.1
clf_name = 'VAE'
clf = VAE(epochs=30, contamination=contamination, gamma=0.8, capacity=0.2)
clf.fit(x_train)

# get the prediction labels and outlier scores of the training data
y_train_pred = clf.labels_  # binary labels (0: inliers, 1: outliers)
y_train_scores = clf.decision_scores_  # raw outlier scores

# get the prediction on the test data
y_test_pred = clf.predict(x_test)  # outlier labels (0 or 1)
y_test_scores = clf.decision_function(x_test)  # outlier scores

# evaluate and print the results
print("\nOn Training Data:")
evaluate_print(clf_name, y_train, y_train_scores)
print("\nOn Test Data:")
evaluate_print(clf_name, y_test, y_test_scores)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 46)]         0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 46)           2162        input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 128)          6016        dense[0][0]                      
__________________________________________________________________________________________________
dropout (Dropout)               (None, 128)          0           dense_1[0][0]                    
______________________________________________________________________________________________

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



81448/94146 [========================>.....] - ETA: 1:00 - loss: 45.9834

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



38901/94146 [===========>..................] - ETA: 4:27 - loss: 45.9814

...

94146/94146 [==============================] - 489s 5ms/step - loss: 45.9978 - val_loss: 46.0206
Epoch 27/30
94146/94146 [==============================] - 481s 5ms/step - loss: 45.9978 - val_loss: 46.0206
Epoch 28/30
33164/94146 [=========>....................] - ETA: 4:54 - loss: 46.0288

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



94146/94146 [==============================] - 462s 5ms/step - loss: 45.9977 - val_loss: 46.0206
Epoch 30/30
94146/94146 [==============================] - 475s 5ms/step - loss: 45.9978 - val_loss: 46.0206


/home/mqli/venv/lib/python3.6/site-packages/sklearn/utils/validation.py:933: FutureWarning: Passing attributes to check_is_fitted is deprecated and will be removed in 0.23. The attributes argument is ignored.
  "argument is ignored.", FutureWarning)
/home/mqli/venv/lib/python3.6/site-packages/sklearn/utils/validation.py:933: FutureWarning: Passing attributes to check_is_fitted is deprecated and will be removed in 0.23. The attributes argument is ignored.
  "argument is ignored.", FutureWarning)



On Training Data:
VAE ROC:0.5774, precision @ rank n:0.1289

On Test Data:
VAE ROC:0.5787, precision @ rank n:0.1294


从ROC指标来看，跟上一个模型效果差不多，有可能是由于时间的原因，训练不够充分，也有可能是因为特征选取的不够准确。